# Sheet properties

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%matplotlib inline
# produce vector inline graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
import matplotlib.pyplot as plt

# Start with local

In [ ]:
import sys
from IPython.display import HTML
sys.path.append('/home/rcendre/classification')

# Imports

In [ ]:
import webbrowser
from pathlib import Path
from IPython.display import display
from numpy import array, logspace
from misvm import SIL, MISVM
from scipy.stats import randint as randint
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.IO import dermatology
from toolbox.transforms.common import PredictorTransform, FlattenTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews, PatchViews
from toolbox.models.models import CustomMIL
from toolbox.transforms.common import BagScaler
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [ ]:
# Advanced parameters
data_type = 'Full'
validation = 4
settings = Settings.get_default_dermatology()

In [ ]:
extractor = 'Wiltgen'

# Inputs

In [ ]:
inputs = IO.load(f'Features.pickle')

In [ ]:
inputs = inputs[(inputs['Binary_Diagnosis'] == 'Benign') |(inputs['Binary_Diagnosis'] == 'Malignant')].reset_index(drop=True)

# Model evaluation

## Models

In [ ]:
models = []
models.append(('SIL', 
               CustomMIL(SIL(kernel='linear', C=1.0, verbose=False)),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))
models.append(('MISVM', 
               CustomMIL(MISVM(kernel='linear', C=1.0, verbose=False)),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))

## Evaluation

In [ ]:
weak_file = f'Weak_Transfer.pickle'

In [ ]:
low_folds = [([1], [2,3]),
             ([2], [3,4]),
             ([3], [4,1]),
             ([4], [1,2])]
Tools.fit_predict(inputs, {'datum': extractor, 'label_encode': 'DiagnosisEncode'}, StandardScaler(), 'Scaled', folds=low_folds)

# Simple labels
all_image = [True] * len(inputs.index)
single_image = inputs['ID_Image'] == '0M'
Data.build_bags(inputs, single_image, 'Reference', all_image, 'Reference', extractor)
Data.build_bags(inputs, single_image, 'Reference', all_image, 'Reference', 'Scaled')
inputs = inputs[single_image].reset_index()

In [ ]:
high_folds = [([2,3], [4]),
             ([3,4], [1]),
             ([4,1], [2]),
             ([1,2], [3])]
for model in models:
    model_name, model_kernel,model_params = model
    # MIL evaluation
    Tools.evaluate(inputs, {'datum': extractor, 'label_encode': 'DiagnosisEncode'}, model_kernel, model_name, distribution=model_params, folds=high_folds)

    # Save
    IO.save(inputs, weak_file)
    
for model in models:
    model_name, model_kernel,model_params = model
    # MIL evaluation
    Tools.evaluate(inputs, {'datum': 'Scaled', 'label_encode': 'DiagnosisEncode'}, model_kernel, f'Scale{model_name}', distribution=model_params, folds=high_folds)

    # Save
    IO.save(inputs, weak_file)

## Scores and ROC

In [28]:
inputs = IO.load(weak_file)

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))
    
for model in models:  
    # Decompose model
    model_name, model_process, model_params = model        
    
    name = model_name
    # Label
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'DiagnosisEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))
    
    name = f'Scale{model_name}'
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'DiagnosisEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))

,f1-score,precision,recall,support
Benign,0.67±0.15,0.69±0.13,0.66±0.17,114.00±3.35
Malignant,0.74±0.10,0.73±0.12,0.75±0.09,138.00±1.80
accuracy,0.71±0.12,0.71±0.12,0.71±0.12,0.71±0.12
macro avg,0.71±0.12,0.71±0.12,0.71±0.12,252.00±4.24
weighted avg,0.71±0.12,0.71±0.12,0.71±0.12,252.00±4.24


,f1-score,precision,recall,support
Benign,0.67±0.10,0.65±0.09,0.69±0.12,114.00±3.35
Malignant,0.71±0.08,0.73±0.08,0.70±0.08,138.00±1.80
accuracy,0.69±0.08,0.69±0.08,0.69±0.08,0.69±0.08
macro avg,0.69±0.08,0.69±0.08,0.69±0.08,252.00±4.24
weighted avg,0.70±0.08,0.70±0.08,0.69±0.08,252.00±4.24


,f1-score,precision,recall,support
Benign,0.67±0.15,0.69±0.13,0.66±0.17,114.00±3.35
Malignant,0.74±0.10,0.73±0.12,0.75±0.09,138.00±1.80
accuracy,0.71±0.12,0.71±0.12,0.71±0.12,0.71±0.12
macro avg,0.71±0.12,0.71±0.12,0.71±0.12,252.00±4.24
weighted avg,0.71±0.12,0.71±0.12,0.71±0.12,252.00±4.24
